In [1]:
!  pip install -q pandas matplotlib torch evaluate transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
from pathlib import Path

import pandas as pd
import numpy as np
import sklearn

import torch
from torch.nn.functional import normalize

import evaluate
from datasets import Dataset, DatasetDict, load_metric
from transformers import (AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments)

In [3]:
from sklearn.model_selection import train_test_split

df_train_emotion = pd.read_csv('train_transcription_df.csv').rename(columns={'Vosk ASR':'text', 'Emotion':'label'})
df_dev_emotion = pd.read_csv('dev_transcription_df.csv').rename(columns={'Vosk ASR':'text', 'Emotion':'label'})
df_test_emotion = pd.read_csv('test_transcription_df.csv').rename(columns={'Vosk ASR':'text', 'Emotion':'label'})

In [4]:
pd.DataFrame([len(df_train_emotion['text'][idx].split() if type(df_train_emotion['text'][idx])==str else []) for idx in df_train_emotion.index]).describe()

,0
count,9989.000000
mean,7.293223
std,6.411954
min,0.000000
25%,2.000000
50%,6.000000
75%,10.000000
max,62.000000


In [5]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
# model_name = "j-hartmann/emotion-english-distilroberta-base"
# model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)

sentence_max_length = 64

In [15]:
label2id_sentiment = {"negative": 0, "positive": 1, "neutral": 2}
id2label_sentiment = {0:"negative", 1:"positive", 2:"neutral"}

label2id_emotion = {'neutral':0 , 'surprise':1 , 'fear':2, 'sadness':3, 'joy':4, 'disgust':5, 'anger':6}
id2label_emotion = {0:'neutral' , 1:'surprise' , 2:'fear', 3:'sadness', 4:'joy', 5:'disgust', 6:'anger'}

from sklearn.metrics import f1_score

# accuracy = evaluate.load("accuracy")
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    labels = labels.astype(int).tolist()
    predictions = np.argmax(predictions, axis=1).astype(int).tolist()
    return {'wf1': f1_score(labels, predictions, average='weighted')}


def tokenize_function(examples, prediction=False, cuda=False):
    if prediction:
        tokenized = tokenizer(examples['text'], max_length=sentence_max_length, padding="max_length", truncation=True, return_tensors='pt')
    else:
        tokenized = tokenizer(examples['text'], max_length=sentence_max_length, padding="max_length", truncation=True)
    if cuda:
        tokenized_cuda = {}
        for key, value in tokenized.items():
            tokenized_cuda[key] = value.cuda()
        return tokenized_cuda
    else:
        return tokenized


def construct_dataset(df, is_emotion=False):
    new_dataset = []
    print(len(df))
    for _, item  in df.iterrows():
        if type(item['text'])!=str:
            continue
        else:
            new_dataset.append({
                "text" : item['text'],
                "label" : label2id_emotion[item['label']] if is_emotion else 
                            label2id_sentiment[item['label']],
            })
        # new_dataset.append({
        #     "text" : item['text'] if type(item['text'])==str else '',
        #     "label" : label2id_emotion[item['label']] if is_emotion else 
        #                 label2id_sentiment[item['label']] ,
        # })
    print(len(new_dataset))
    new_df = pd.DataFrame(new_dataset)
    new_ds = Dataset.from_pandas(new_df)
    tokenized_ds = new_ds.map(tokenize_function, batched=True)
    return tokenized_ds

In [ ]:
dataset_emotion = DatasetDict()
dataset_emotion['train'] = construct_dataset(df_train_emotion, is_emotion=True)
dataset_emotion['validation'] = construct_dataset(df_dev_emotion, is_emotion=True)
dataset_emotion['test'] = construct_dataset(df_test_emotion, is_emotion=True)

In [ ]:
classifier_model_emotion = AutoModelForSequenceClassification.from_pretrained(
    model_name, # put path instead of model name for pretrained models, 
    num_labels=len(label2id_emotion),
    ignore_mismatched_sizes=True
)
device = torch.device("cuda:0")
classifier_model_emotion.to(device)

In [18]:
num_epochs = 5
training_args = TrainingArguments(
    output_dir='outputs',
    num_train_epochs=num_epochs,
    # per_device_train_batch_size=2,
    # per_device_eval_batch_size=2,
    warmup_steps=50,
    weight_decay=0.01,
    logging_steps=1,
    evaluation_strategy='epoch',
    # evaluation_strategy='steps',
    # eval_steps=100,
    # save_strategy ='steps',
    # save_steps =200,
    save_strategy ='epoch',
    # load_best_model_at_end=True,
    auto_find_batch_size=True,
)

trainer = Trainer(
    model=classifier_model_emotion,
    args=training_args,
    train_dataset=dataset_emotion['train'],
    eval_dataset=dataset_emotion['validation'],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=dataset_emotion['test'])